## Summary of Study Results

With the data downloads from Replica, create a single table with a row for each origin station and columns for “car trips to areas around other destination stations on the other lines” and another column for “train trips”

In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *
import numpy as np

from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis import utils
from calitp_data_analysis import get_fs
import gcsfs as fs
fs = get_fs()

In [2]:
import replica_utils

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/MetroLink_LinkUS/"


In [5]:
### list of station names based on the replica geometry download names
station_geom_list = ["lossan-simi-valley", "lossan-burbank-airport", "lossan-oxnard",  "lossan-camarillo", "lossan-moorpark",  "lossan-chatsworth",
                     "lossan-northridge", "lossan-van-nuys", "lossan-burbank", "lossan-glendale", 
                     "lossan-commerce", "lossan-norwalk", "lossan-buena-park", "lossan-fullerton", "lossan-anaheim",
                     "lossan-orange", "lossan-santa-ana", "lossan-tustin", "lossan-irvine", "lossan-laguna-niguel", 
                     "lossan-san-juan-capistrano", "lossan-san-clemente", "lossan-san-clemente-pier", "lossan-oceanside", 
                     "lossan-solana-beach", "lossan-san-diego", "lossan-ot-san-diego", "lossan-union-station",
                     
                     "riverside-line-montebello-commerce", "riverside-line-industry", "riverside-line-pomona",
                     "riverside-line-ontario","riverside-line-pedley",  "riverside-line-riverside",
                     
                     "sb-line-cal-state-la", "sb-line-el-monte",  "sb-line-baldwin-park", "sb-line-covina",  "sb-line-pomona-north",
                     "sb-line-claremont", "sb-line-montclair", "sb-line-upland", "sb-line-rancho-cucamonga", "sb-line-fontana", 
                     "sb-line-rialto", "sb-line-san-bernardino-depot", "sb-line-san-bernardino-dt", 
                     "sb-line-redlands-dt", "sb-line-redlands-esri", "sb-line-redlands-univ", "sb-line-san-bernardino-tippecanoe",
                     
                     "91-perris-line-corona", "91-perris-line-corona-west", "91-perris-line-moreno-valley-march", 
                     "91-perris-line-perris-south", "91-perris-line-perris-downtown", "91-perris-line-placentia-future", 
                     "91-perris-line-riverside-hunter-park", "91-perris-line-riverside-la-sierra",
                     
                     "antelope-valley-burbank-airport-n", "antelope-valley-lancaster", "antelope-valley-newhall", "antelope-valley-palmdale",
                     "antelope-valley-santa-clarita", "antelope-valley-sun-valley", "antelope-valley-sylmar-san-fernando",
                     "antelope-valley-via-princessa", "antelope-valley-vincent-grade", "antelope-valley-vista-canyon",
                     
                     "inland-empire-oc-anaheim-canyon", 
                     
                     "ventura-line-ventura-east", "ventura-line-ventura-dtwn",                    
                    ]   

In [6]:
#### define the study data download names. Ideally this would be the same as the origin station geometries
analyses_study_data_list = [ "line_origin_union_station",
                         "riversideline_origin_montebello", "riversideline_origin_industry", "riversideline_origin_pomona",  
                         "riversideline_origin_ontario", "riversideline_origin_pedley",  "riversideline_origin_riverside",
                         
                         "sbline_origin_calstate_la", "sbline_origin_el_monte",  "sbline_origin_baldwin_park",
                         "sbline_origin_covina",  "sbline_origin_pomona_north", "sbline_origin_fontana",
                         "sbline_origin_claremont", "sbline_origin_montclair", "sbline_origin_upland", "sbline_origin_rancho_cucamonga", 
                         "sbline_origin_rialto", "sbline_origin_san_bernardino_depot", "sbline_origin_san_bernardino_dt", 
                         
                         "origin_simi_valley", "origin_burbank_airport", "origin_oxnard",  "origin_camarillo", "origin_moorpark",  "origin_chatsworth",
                         "origin_northridge", "origin_van_nuys", "origin_burbank", "origin_glendale", 
                         "origin_commerce", "origin_norwalk", "origin_buena_park", "origin_fullerton", "origin_anaheim",
                         "origin_orange", "origin_santa_ana", "origin_tustin", "origin_irvine", "origin_laguna_niguel", 
                         "origin_san_juan_capistrano", "origin_san_clemente", "origin_san_clemente_pier", "origin_oceanside", 
                         "origin_solana_beach", "origin_san_diego", "origin_old_town_san_diego",

                         "origin_corona", "origin_corona_west", "origin_moreno_valley", 
                         "origin_perris_south", "origin_perris", "origin_placentia", 
                         "origin_riverside_hunter_park", #"origin_riverside_la_sierra",

                         "origin_burbank_airport_north", "origin_lancaster", "origin_newhall", "origin_palmdale",
                         "origin_santa_clarita", "origin_sun_valley", "origin_sylmar",
                         "origin_via_princessa", "origin_vincent_grade", "origin_vista_canyon",

                         "origin_anaheim_canyon", 

                         "origin_ventura_east", "origin_ventura_downtown_beach",        
                         
                        ]

In [7]:
summary = replica_utils.return_score_summary(analyses_study_data_list, station_geom_list)

In [8]:
summary

,station_name,station_name_full,metrolink_lines,total_trips_from_origin_station,n_auto_trips_to_other_station_areas,pct_auto_trips_to_other_station_areas,n_tranist_trips_to_other_station_areas,pct_transit_trips_to_other_station_areas,auto_mean_minutes,auto_median_minutes,auto_max_minutes,auto_mean_miles,auto_median_miles,auto_max_miles,transit_mean_minutes,transit_median_minutes,transit_max_minutes,transit_mean_miles,transit_median_miles,transit_max_miles,auto_top_tract_traveled_to,auto_least_tract_traveled_to,transit_top_tract_traveled_to,transit_least_tract_traveled_to
0,UNION STATION,LA Union Station,"Antelope Valley Line, Orange County Line, Rive...",18657,10339,0.554162,938,0.050276,31.680143,24.0,179,21.165722,14.3,123.8,78.638593,64.0,1434,21.760768,21.10,74.2,Cal State LA,Solana Beach,Glendale,Corona - West
1,MONTEBELLO,Montebello/Commerce,Riverside Line,42917,22560,0.525666,655,0.015262,13.369238,7.0,145,7.194459,2.7,117.5,42.964885,30.0,217,7.341221,3.50,84.0,Montebello/Commerce,Redlands - Esri,Montebello/Commerce,Irvine
2,INDUSTRY,Industry,Riverside Line,8588,5436,0.632976,51,0.005939,26.223142,23.0,128,17.304213,13.9,112.2,97.647059,82.0,211,26.864706,23.90,66.8,Pomona - North,San Diego,LA Union Station,Commerce
3,POMONA,Pomona - Downtown,Riverside Line,19454,10982,0.564511,186,0.009561,15.845110,11.0,107,9.885230,5.4,98.3,58.634409,47.0,237,13.086559,10.60,89.5,Pomona - North,Simi Valley,Pomona - North,Redlands - University
4,ONTARIO,Ontario - East,San Bernardino Line,63621,35456,0.557300,74,0.001163,18.636084,16.0,140,11.136967,9.1,124.2,84.351351,82.5,187,17.785135,16.00,47.7,Rancho Cucamonga,Solana Beach,Rancho Cucamonga,San Bernardino - Tippecanoe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,VINCENT GRADE,Vincent Grade/Acton,Antelope Valley Line,6609,3532,0.534423,1,0.000151,22.870895,18.0,186,17.500651,14.0,100.6,45.000000,45.0,45,10.000000,10.00,10.0,Palmdale,Burbank Airport,Palmdale,Palmdale
63,VISTA CANYON,Vista Canyon,Antelope Valley Line,38595,24929,0.645913,218,0.005648,9.828393,7.0,143,6.172321,3.5,102.1,36.894495,26.0,203,7.613303,2.85,80.8,Via Princessa,Corona - North Main,Via Princessa,Glendale
64,ANAHEIM CANYON,Anaheim Canyon,Inland Empire-Orange County Line,46536,26235,0.563757,208,0.004470,12.597103,10.0,140,7.972346,5.4,116.4,61.225962,47.0,207,11.703365,7.05,70.5,Placentia,Camarillo,Placentia,Burbank - Downtown
65,VENTURA EAST,Ventura - East,Ventura County Line,35910,22712,0.632470,261,0.007268,13.641555,10.0,254,9.180653,5.8,233.4,42.015326,37.0,129,5.478927,5.10,18.3,Ventura - Downtown / Beach,Anaheim Canyon,Oxnard,Ventura - Downtown / Beach


### Exporting the file locally for now

In [9]:
## export
# result_summary.to_csv("MetroLink_Stations_Replica_Summaries_072225.csv")